# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from datetime import datetime as dt
import json

# Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: - - .

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: - - - <текст статьи>



In [2]:
KEYWORDS = ['python', 'парсинг', 'php']


In [3]:
start_time = dt.now()
dict_url = []
news_filtr = []
x = 1
while x < 10:
    url = 'https://habr.com/ru/all/page' + str(x)
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        if 'сегодня' in post.find('span', {'class': 'post__time'}).text:
            url_news = post.find('a', {'class': 'post__title_link'}).get('href')
            dict_url.append(url_news)
        else:
            x = 10
            break        
    x += 1
    time.sleep(0.3)
dict_url
for news in dict_url:
    url = news
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1', {'class' : 'post__title post__title_full'}).text
    posts = soup.find('div', {'class' : 'post__body post__body_full'})   
    for key in KEYWORDS:
        if key in posts.text.lower().split():
            news_filtr.append({'keyword': key, 'url_news' : url, 'title_news' : title, 'post' : post})
        else:
            continue
    time.sleep(0.3)
df_news = pd.DataFrame(news_filtr)
print(dt.now() - start_time)  
df_news
    

0:00:07.097320


,keyword,url_news,title_news,post
0,php,https://habr.com/ru/post/522348/,"\nТипобезопасная работа с массивами PHP, часть...","[\n, [\n, [\n, <span class=""default-image_mini..."


# Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>.

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.



In [4]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [5]:
dict_avast = []
for email in EMAIL:
    url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data?'
    params = {"emailAddresses":[email]}
    headers = {'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'}
    res = requests.post(url, data=json.dumps(params), headers = headers)
    for x in res.json()['breaches']:
        dict_avast.append({'email': email, 'publishDate': res.json()['breaches'][x]['publishDate'], 'site' : res.json()['breaches'][x]['site'], 'description' : res.json()['breaches'][x]['description']})
    df_avast = pd.DataFrame(dict_avast)
df_avast.head()

,email,publishDate,site,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
